# Section 1 - imports

In [1]:
import pandas as pd
import requests
import io
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

# bigquery

In [18]:
import google.auth

credentials, project = google.auth.default()

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [56]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
)

In [57]:
from google.cloud import bigquery

client = bigquery.Client(credentials=credentials)

In [39]:
datasets = list(client.list_datasets())

In [42]:
for dataset in datasets:
    print(dataset.dataset_id)

bt_syn_individual_test
dbt_emason
elh_pdg_elmo
iris
jaffle_shop
pokemon_api
stripe


In [58]:
tables = list(client.list_tables("iris"))

In [60]:
for table in tables:
    print(table.table_id)

iris_data


dataframe.to_gbq

In [111]:
c.to_gbq("iris.iris_data", credentials=credentials, if_exists="append")

100%|██████████| 1/1 [00:00<00:00, 1695.35it/s]


# section 2 - API calls

In [4]:
response = requests.get("https://pokeapi.co/api/v2/pokemon?limit=2000")

In [5]:
response.json()

{'count': 1302,
 'next': None,
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
  {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
  {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
  {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
  {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
  {

In [6]:
df_pokemon = pd.json_normalize(response.json()["results"])

In [7]:
response_type = requests.get("https://pokeapi.co/api/v2/type")

In [8]:
df_types = pd.DataFrame(response_type.json()["results"])

In [9]:
response = requests.get(f"https://pokeapi.co/api/v2/type/{df_types.iloc[0,0]}")

In [10]:
pd.json_normalize(response.json()["pokemon"])

,slot,pokemon.name,pokemon.url
0,1,pidgey,https://pokeapi.co/api/v2/pokemon/16/
1,1,pidgeotto,https://pokeapi.co/api/v2/pokemon/17/
2,1,pidgeot,https://pokeapi.co/api/v2/pokemon/18/
3,1,rattata,https://pokeapi.co/api/v2/pokemon/19/
4,1,raticate,https://pokeapi.co/api/v2/pokemon/20/
...,...,...,...
153,1,squawkabilly-yellow-plumage,https://pokeapi.co/api/v2/pokemon/10261/
154,1,squawkabilly-white-plumage,https://pokeapi.co/api/v2/pokemon/10262/
155,2,ursaluna-bloodmoon,https://pokeapi.co/api/v2/pokemon/10272/
156,1,terapagos-terastal,https://pokeapi.co/api/v2/pokemon/10276/


In [11]:
df_append = pd.DataFrame()
for pokemon_type in df_types["name"]:
    print(pokemon_type)
    response = requests.get(f"https://pokeapi.co/api/v2/type/{pokemon_type}")
    df_temp = pd.json_normalize(response.json()["pokemon"])
    df_temp["type"] = pokemon_type
    df_append = pd.concat([df_append, df_temp], axis=0)

normal
fighting
flying
poison
ground
rock
bug
ghost
steel
fire
water
grass
electric
psychic
ice
dragon
dark
fairy
unknown
shadow


In [12]:
df_append.loc[df_append["pokemon.name"].str.contains("ratt")]

,slot,pokemon.name,pokemon.url,type
3,1.0,rattata,https://pokeapi.co/api/v2/pokemon/19/,normal
136,2.0,rattata-alola,https://pokeapi.co/api/v2/pokemon/10091/,normal
76,1.0,rattata-alola,https://pokeapi.co/api/v2/pokemon/10091/,dark


In [13]:
response = requests.get("https://pokeapi.co/api/v2/region/")

In [14]:
response.status_code

200

In [15]:
response.json()

{'count': 10,
 'next': None,
 'previous': None,
 'results': [{'name': 'kanto', 'url': 'https://pokeapi.co/api/v2/region/1/'},
  {'name': 'johto', 'url': 'https://pokeapi.co/api/v2/region/2/'},
  {'name': 'hoenn', 'url': 'https://pokeapi.co/api/v2/region/3/'},
  {'name': 'sinnoh', 'url': 'https://pokeapi.co/api/v2/region/4/'},
  {'name': 'unova', 'url': 'https://pokeapi.co/api/v2/region/5/'},
  {'name': 'kalos', 'url': 'https://pokeapi.co/api/v2/region/6/'},
  {'name': 'alola', 'url': 'https://pokeapi.co/api/v2/region/7/'},
  {'name': 'galar', 'url': 'https://pokeapi.co/api/v2/region/8/'},
  {'name': 'hisui', 'url': 'https://pokeapi.co/api/v2/region/9/'},
  {'name': 'paldea', 'url': 'https://pokeapi.co/api/v2/region/10/'}]}

In [16]:
response = requests.get("https://pokeapi.co/api/v2/region/johto")

In [26]:
response.json()

{'id': 2,
 'locations': [{'name': 'blackthorn-city',
   'url': 'https://pokeapi.co/api/v2/location/65/'},
  {'name': 'burned-tower', 'url': 'https://pokeapi.co/api/v2/location/66/'},
  {'name': 'cherrygrove-city',
   'url': 'https://pokeapi.co/api/v2/location/69/'},
  {'name': 'cianwood-city', 'url': 'https://pokeapi.co/api/v2/location/70/'},
  {'name': 'dark-cave', 'url': 'https://pokeapi.co/api/v2/location/72/'},
  {'name': 'dragons-den', 'url': 'https://pokeapi.co/api/v2/location/74/'},
  {'name': 'ecruteak-city', 'url': 'https://pokeapi.co/api/v2/location/75/'},
  {'name': 'ice-path', 'url': 'https://pokeapi.co/api/v2/location/77/'},
  {'name': 'ilex-forest', 'url': 'https://pokeapi.co/api/v2/location/78/'},
  {'name': 'lake-of-rage', 'url': 'https://pokeapi.co/api/v2/location/79/'},
  {'name': 'mt-mortar', 'url': 'https://pokeapi.co/api/v2/location/81/'},
  {'name': 'mt-silver', 'url': 'https://pokeapi.co/api/v2/location/82/'},
  {'name': 'national-park', 'url': 'https://pokeapi.c

In [18]:
pd.json_normalize(response.json()['pokedexes'])

,name,url
0,original-johto,https://pokeapi.co/api/v2/pokedex/3/
1,updated-johto,https://pokeapi.co/api/v2/pokedex/7/


In [23]:
pd.json_normalize(response.json()['locations'])

,name,url
0,blackthorn-city,https://pokeapi.co/api/v2/location/65/
1,burned-tower,https://pokeapi.co/api/v2/location/66/
2,cherrygrove-city,https://pokeapi.co/api/v2/location/69/
3,cianwood-city,https://pokeapi.co/api/v2/location/70/
4,dark-cave,https://pokeapi.co/api/v2/location/72/
...,...,...
61,pokewalker,https://pokeapi.co/api/v2/location/249/
62,cliff-edge-gate,https://pokeapi.co/api/v2/location/250/
63,radio-tower,https://pokeapi.co/api/v2/location/252/
64,roaming-johto,https://pokeapi.co/api/v2/location/800/


In [25]:
pd.json_normalize(response.json()['names'])

,name,language.name,language.url
0,ジョウト,ja-Hrkt,https://pokeapi.co/api/v2/language/1/
1,성도,ko,https://pokeapi.co/api/v2/language/3/
2,城都,zh-Hant,https://pokeapi.co/api/v2/language/4/
3,Johto,fr,https://pokeapi.co/api/v2/language/5/
4,Johto,de,https://pokeapi.co/api/v2/language/6/
5,Johto,it,https://pokeapi.co/api/v2/language/8/
6,Johto,en,https://pokeapi.co/api/v2/language/9/
7,城都,zh-Hans,https://pokeapi.co/api/v2/language/12/


In [46]:
response = requests.get("https://pokeapi.co/api/v2/pokemon-species")

In [47]:
response.status_code

200

In [48]:
response.json()

{'count': 1025,
 'next': 'https://pokeapi.co/api/v2/pokemon-species?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon-species/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon-species/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon-species/3/'},
  {'name': 'charmander',
   'url': 'https://pokeapi.co/api/v2/pokemon-species/4/'},
  {'name': 'charmeleon',
   'url': 'https://pokeapi.co/api/v2/pokemon-species/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon-species/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon-species/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon-species/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon-species/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon-species/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon-species/11

# Section 3 - DuckDB testing

In [3]:
import duckdb

In [4]:
duck_conn = duckdb.connect("../../data/db/pokemon.duckdb")

In [ ]:
duck_conn.close()

In [76]:
duck_conn.sql("show all tables").fetchall()

[('pokemon',
  'main',
  'pokemon',
  ['name', 'url'],
  ['VARCHAR', 'VARCHAR'],
  False),
 ('pokemon',
  'main',
  'pokemon_query',
  ['name', 'url'],
  ['VARCHAR', 'VARCHAR'],
  False),
 ('pokemon',
  'main',
  'pokemon_types',
  ['name', 'url'],
  ['VARCHAR', 'VARCHAR'],
  False),
 ('pokemon',
  'main',
  'pokemon_types_query',
  ['name', 'url'],
  ['VARCHAR', 'VARCHAR'],
  False)]

In [31]:
duck_conn.execute("select * from pokemon limit 10;").fetchall()

[('bulbasaur', 'https://pokeapi.co/api/v2/pokemon/1/'),
 ('ivysaur', 'https://pokeapi.co/api/v2/pokemon/2/'),
 ('venusaur', 'https://pokeapi.co/api/v2/pokemon/3/'),
 ('charmander', 'https://pokeapi.co/api/v2/pokemon/4/'),
 ('charmeleon', 'https://pokeapi.co/api/v2/pokemon/5/'),
 ('charizard', 'https://pokeapi.co/api/v2/pokemon/6/'),
 ('squirtle', 'https://pokeapi.co/api/v2/pokemon/7/'),
 ('wartortle', 'https://pokeapi.co/api/v2/pokemon/8/'),
 ('blastoise', 'https://pokeapi.co/api/v2/pokemon/9/'),
 ('caterpie', 'https://pokeapi.co/api/v2/pokemon/10/')]

In [9]:
duck_conn.close()

# example to build tables using previous endpoint responses as iterables

In [53]:
# pokemon_response = requests.get("https://pokeapi.co/api/v2/pokemon?limit=2000")
df_pokemon_append = pd.DataFrame()

pokemon_names_list = duck_conn.execute("select name from pokemon;").fetchall()
for pokemon in pokemon_names_list:
    pokemon_response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{pokemon[0]}")

    temp_df = pd.json_normalize(pokemon_response.json())

    df_pokemon_append = pd.concat([df_pokemon_append, temp_df], axis=0)

## Main pokemon call (async)

In [11]:
async def fetch_pokemon(session, pokemon_name):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_name}"
    async with session.get(url, ssl=False) as response:
        return await response.json()

In [12]:
import asyncio
import aiohttp

In [13]:

pokemon_names_list = duck_conn.execute("select name from pokemon;").fetchall()

In [16]:
async with aiohttp.ClientSession() as session:
        tasks = [fetch_pokemon(session, pokemon_name[0]) for pokemon_name in pokemon_names_list]
        pokemon_responses = await asyncio.gather(*tasks)

In [19]:
df_async_pokemon_response = pd.json_normalize(pokemon_responses)

In [20]:
df_async_pokemon_response

,abilities,base_experience,forms,game_indices,height,held_items,id,is_default,location_area_encounters,moves,...,sprites.versions.generation-vi.x-y.front_shiny,sprites.versions.generation-vi.x-y.front_shiny_female,sprites.versions.generation-vii.icons.front_default,sprites.versions.generation-vii.icons.front_female,sprites.versions.generation-vii.ultra-sun-ultra-moon.front_default,sprites.versions.generation-vii.ultra-sun-ultra-moon.front_female,sprites.versions.generation-vii.ultra-sun-ultra-moon.front_shiny,sprites.versions.generation-vii.ultra-sun-ultra-moon.front_shiny_female,sprites.versions.generation-viii.icons.front_default,sprites.versions.generation-viii.icons.front_female
0,"[{'ability': {'name': 'overgrow', 'url': 'http...",64.0,"[{'name': 'bulbasaur', 'url': 'https://pokeapi...","[{'game_index': 153, 'version': {'name': 'red'...",7,[],1,True,https://pokeapi.co/api/v2/pokemon/1/encounters,"[{'move': {'name': 'razor-wind', 'url': 'https...",...,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None
1,"[{'ability': {'name': 'overgrow', 'url': 'http...",142.0,"[{'name': 'ivysaur', 'url': 'https://pokeapi.c...","[{'game_index': 9, 'version': {'name': 'red', ...",10,[],2,True,https://pokeapi.co/api/v2/pokemon/2/encounters,"[{'move': {'name': 'swords-dance', 'url': 'htt...",...,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None
2,"[{'ability': {'name': 'overgrow', 'url': 'http...",263.0,"[{'name': 'venusaur', 'url': 'https://pokeapi....","[{'game_index': 154, 'version': {'name': 'red'...",20,[],3,True,https://pokeapi.co/api/v2/pokemon/3/encounters,"[{'move': {'name': 'swords-dance', 'url': 'htt...",...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,None
3,"[{'ability': {'name': 'blaze', 'url': 'https:/...",62.0,"[{'name': 'charmander', 'url': 'https://pokeap...","[{'game_index': 176, 'version': {'name': 'red'...",6,[],4,True,https://pokeapi.co/api/v2/pokemon/4/encounters,"[{'move': {'name': 'mega-punch', 'url': 'https...",...,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None
4,"[{'ability': {'name': 'blaze', 'url': 'https:/...",142.0,"[{'name': 'charmeleon', 'url': 'https://pokeap...","[{'game_index': 178, 'version': {'name': 'red'...",11,[],5,True,https://pokeapi.co/api/v2/pokemon/5/encounters,"[{'move': {'name': 'mega-punch', 'url': 'https...",...,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None,https://raw.githubusercontent.com/PokeAPI/spri...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,"[{'ability': {'name': 'water-absorb', 'url': '...",NaN,"[{'name': 'ogerpon-wellspring-mask', 'url': 'h...",[],12,[],10273,False,https://pokeapi.co/api/v2/pokemon/10273/encoun...,"[{'move': {'name': 'swords-dance', 'url': 'htt...",...,None,None,None,None,None,None,None,None,None,None
1298,"[{'ability': {'nam

In [55]:
df_pokemon_append = df_pokemon_append.reset_index(drop=True)

In [57]:
df_pokemon_names = duck_conn.execute("select name from pokemon;").fetchall()

In [61]:
df_pokemon_append.to_pickle('df_all_pokemon.pkl')

In [39]:
len(df_pokemon_append.columns)

139

In [50]:
for col in df_pokemon_append.columns:
    print(col)

abilities
base_experience
forms
game_indices
height
held_items
id
is_default
location_area_encounters
moves
name
order
past_abilities
past_types
stats
types
weight
cries.latest
cries.legacy
species.name
species.url
sprites.back_default
sprites.back_female
sprites.back_shiny
sprites.back_shiny_female
sprites.front_default
sprites.front_female
sprites.front_shiny
sprites.front_shiny_female
sprites.other.dream_world.front_default
sprites.other.dream_world.front_female
sprites.other.home.front_default
sprites.other.home.front_female
sprites.other.home.front_shiny
sprites.other.home.front_shiny_female
sprites.other.official-artwork.front_default
sprites.other.official-artwork.front_shiny
sprites.other.showdown.back_default
sprites.other.showdown.back_female
sprites.other.showdown.back_shiny
sprites.other.showdown.back_shiny_female
sprites.other.showdown.front_default
sprites.other.showdown.front_female
sprites.other.showdown.front_shiny
sprites.other.showdown.front_shiny_female
sprites.vers

columns of interest:
abilities, base_experience, forms
game_indicies, height, held_items (Boolean?)
id, moves, 
stats, types, weight

In [101]:
# how to switch row values from a column to column indices

pd.json_normalize(df_pokemon_append.iloc[1:2, 14:15].values[0][0]).pivot_table(
    "base_stat", None, "stat.name"
)

stat.name,attack,defense,hp,special-attack,special-defense,speed
base_stat,62,63,60,80,80,60


In [86]:
pd.json_normalize(df_pokemon_append.iloc[1:2, 14:15].values[0][0])

,base_stat,effort,stat.name,stat.url
0,60,0,hp,https://pokeapi.co/api/v2/stat/1/
1,62,0,attack,https://pokeapi.co/api/v2/stat/2/
2,63,0,defense,https://pokeapi.co/api/v2/stat/3/
3,80,1,special-attack,https://pokeapi.co/api/v2/stat/4/
4,80,1,special-defense,https://pokeapi.co/api/v2/stat/5/
5,60,0,speed,https://pokeapi.co/api/v2/stat/6/


# TODO

abilities

In [33]:
df_pokemon_append['abilities'].iloc[4] # ability is nested within an array

[{'ability': {'name': 'blaze', 'url': 'https://pokeapi.co/api/v2/ability/66/'},
  'is_hidden': False,
  'slot': 1},
 {'ability': {'name': 'solar-power',
   'url': 'https://pokeapi.co/api/v2/ability/94/'},
  'is_hidden': True,
  'slot': 3}]

In [51]:
df_append_ability = pd.DataFrame()

for i in range(len(df_pokemon_append)):
    temp_df = pd.json_normalize(
        df_pokemon_append['abilities'].iloc[i]
        )
    temp_df['pokemon_name'] = df_pokemon_append['name'].iloc[i]
    temp_df['pokemon_id'] = df_pokemon_append['id'].iloc[i]
    df_append_ability = pd.concat([df_append_ability, temp_df], axis=0)


In [52]:
df_append_ability

,is_hidden,slot,ability.name,ability.url,pokemon_name,pokemon_id
0,False,1,overgrow,https://pokeapi.co/api/v2/ability/65/,bulbasaur,1
1,True,3,chlorophyll,https://pokeapi.co/api/v2/ability/34/,bulbasaur,1
0,False,1,overgrow,https://pokeapi.co/api/v2/ability/65/,ivysaur,2
1,True,3,chlorophyll,https://pokeapi.co/api/v2/ability/34/,ivysaur,2
0,False,1,overgrow,https://pokeapi.co/api/v2/ability/65/,venusaur,3
1,True,3,chlorophyll,https://pokeapi.co/api/v2/ability/34/,venusaur,3
0,False,1,blaze,https://pokeapi.co/api/v2/ability/66/,charmander,4
1,True,3,solar-power,https://pokeapi.co/api/v2/ability/94/,charmander,4
0,False,1,blaze,https://pokeapi.co/api/v2/ability/66/,charmeleon,5
1,True,3,solar-power,https://pokeapi.co/api/v2/ability/94/,charmeleon,5


base_experience

forms


game_indices

height


held_items

moves

stats

types

weight

In [49]:
pokemon_response = requests.get("https://pokeapi.co/api/v2/pokemon/bulbasaur")

In [68]:
pokemon_response.json()['moves'][0]['move']

{'name': 'razor-wind', 'url': 'https://pokeapi.co/api/v2/move/13/'}

In [71]:
pd.json_normalize(pokemon_response.json()['moves'][0])

0    razor-wind
Name: move.name, dtype: object

In [40]:
duck_conn.close()